# RA2CE feature: Losses analysis

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import rasterio
from branca.colormap import LinearColormap
import folium

from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

root_dir = Path(".\\data\\losses")
assert root_dir.exists(), "root_dir not found."

_network_ini_name = "network_losses.ini" #set the name for the network.ini
_analysis_ini_name = "analysis_losses.ini" #set the name for the analysis.ini

network_ini_losses = root_dir / _network_ini_name
assert network_ini_losses.is_file()

analysis_ini_losses = root_dir / _analysis_ini_name
assert analysis_ini_losses.is_file()

c:\repos\ra2ce\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set the paths to the initialization files and check if the files exist.

In [ ]:
_network_ini_name = "network_criticality.ini" #set the name for the network.ini
_analysis_ini_name = "analysis_criticality.ini" #set the name for the analysis.ini

network_ini_criticality = root_dir / _network_ini_name
assert network_ini_criticality.is_file()

analysis_ini_criticality = root_dir / _analysis_ini_name
assert analysis_ini_criticality.is_file()

Run RA2CE.

In [ ]:
handler = Ra2ceHandler(network=network_ini_criticality, analysis=analysis_ini_criticality)
criticality_analysis_weight = handler.input_config.analysis_config.config_data.indirect[0].weighing.config_value

handler.configure()
handler.run_analysis()

#### Visualising the network

In [ ]:
# Set the path to your output_graph folder to find the network/graph creation:
path_output_graph = root_dir / "static" / "output_graph"

# Now we find and inspect the file 'base_graph_edges.gpkg' which holds the 'edges' of the network. 
# An edge (or link) of a network (or graph) represents a connection between two nodes (or vertices) of the network. More information on: https://mathinsight.org/definition/network_edge#:~:text=An%20edge%20(or%20link)%20of,in%20the%20first%20figure%20below.
base_graph_edges = path_output_graph / "base_network.gpkg"
edges_gdf = gpd.read_file(base_graph_edges, driver = "GPKG")
edges_gdf.head()

edges_gdf.explore(tiles="CartoDB positron", popup = True)

In [ ]:
#  Inspect the criticality analysis results

# Set the path to your output folder:
path_output = root_dir / "output" / "single_link_redundancy"
redundancy_file = path_output / "republic_dominican_redundancy.gpkg"
redundancy_gdf = gpd.read_file(redundancy_file, driver = "GPKG")

redundancy_gdf.explore(tiles="CartoDB positron", popup = True, column = f"alt_{criticality_analysis_weight}")

## Losses Analysis

In [2]:
handler = Ra2ceHandler(network=network_ini_losses, analysis=analysis_ini_losses)
criticality_analysis_weight = handler.input_config.analysis_config.config_data.indirect[0].weighing.config_value

handler.configure()
handler.run_analysis()

2024-03-25 11:39:08 AM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.


2024-03-25 11:39:08 AM - [networks.py:186] - root - INFO - Apparently, you already did create a network with ra2ce earlier. Ra2ce will use this: data\losses\static\output_graph\base_graph.p
2024-03-25 11:39:08 AM - [hazard_overlay.py:108] - root - INFO - Initialized hazard object.
2024-03-25 11:39:08 AM - [hazard_overlay.py:696] - root - INFO - Iterating overlay of GeoPandas Dataframe.
2024-03-25 11:39:08 AM - [hazard_overlay.py:734] - root - INFO - Setting 'base_network_hazard' graph.
2024-03-25 11:39:08 AM - [hazard_intersect_builder_for_tif.py:183] - root - WARNING - Some geometries have NoneType objects (no coordinate information), namely: Empty GeoDataFrame
Columns: [ID, link_type, avgspeed, geometry, node_A, node_B, edge_fid, length]
Index: [].This could be due to segmentation, and might cause an exception in hazard overlay
Network hazard overlay with Inundacion_Fluvial_tr100: 100%|██████████| 789/789 [00:07<00:00, 105.33it/s]
Network fraction with hazard overlay with Inundacion_

#### Introducing a hazardous event

In [ ]:
hazard_folder = root_dir / "static" / "hazard" # find the hazard folder where you locate your floo dmap
hazard_map = hazard_folder / "Inundacion_Fluvial_tr1000.tif" # set the location of the hazard map

# Open the TIF file using rasterio
with rasterio.open(hazard_map) as src:
    # Read the TIF file as a numpy array
    tif_array = src.read(1)  # Change the band index (1) if necessary

plt.figure(figsize=(10, 10))
plt.imshow(tif_array, cmap='Blues')  # Change the colormap if desired
plt.colorbar(label='Pixel Values')
plt.title('Flood map')
plt.show()


In [ ]:
# Show inundated links

inundation_threshold = 0.5

hazard_output = root_dir / "static" / "output_graph" / "base_graph_hazard_edges.gpkg"
hazard_gdf = gpd.read_file(hazard_output, driver = "GPKG")

# Create a colormap
colormap = LinearColormap(['lightblue', 'blue', 'darkblue'], vmin=inundation_threshold, vmax=hazard_gdf['EV1_ma'].max(), caption='EV1_ma')

center = [hazard_gdf['geometry'].centroid.y.mean(), hazard_gdf['geometry'].centroid.x.mean()]
m = folium.Map(location=center, zoom_start=12, control_scale=True, tiles="cartodbpositron")

# Create two separate FeatureGroup layers
gray_layer = folium.FeatureGroup(name='Unimpacted links', show=True)
colored_layer = folium.FeatureGroup(name='Inundated links', overlay=True, control=False)

# Add edges to the m
for idx, row in hazard_gdf.iterrows():
    color = 'gray' if (row['EV1_ma'] < inundation_threshold) or ('bridge' in row and row['bridge'] == 'yes') else colormap(row['EV1_ma'])
    
    # Extracting coordinates from LineString
    coordinates = [(coord[1], coord[0]) for coord in row['geometry'].coords]

    # Create a popup with data
    popup_content = f"<strong>Water height(m):</strong> {round(row['EV1_ma'], 1)}"
    popup = folium.Popup(popup_content, max_width=300)
    
    # Add PolyLine to the appropriate layer
    folium.PolyLine(
        locations=coordinates,
        color=color,
        weight=2,
        opacity=1,
        popup=popup
    ).add_to(gray_layer if row['EV1_ma'] < inundation_threshold else colored_layer)

# Add FeatureGroup layers to the m
gray_layer.add_to(m)
colored_layer.add_to(m)

# Add colormap to the m
colormap.add_to(m)

# Add LayerControl to the m
folium.LayerControl().add_to(m)

# Display the m
m